In [1]:
import numpy as np
import random
from math import inf

In [2]:
def get_user_input(size):
    i,j = map(int,input(f"enter row and column index from [0,1,2,...{size-1}]->"
                        "should be comma separated : ").split(','))
    print(f"i : {i} and j is {j}")
    return i,j

In [3]:
def getAllWindows(playGround,size,connect):
    windows = []
    for i in range(size-connect+1):
        for j in range(size - connect + 1):
            windows.append(playGround[i:i+connect,j:j+connect])
    return windows
            

In [4]:
def checkRowColumn(subPlayGround,size):
    p = [('x',-1),('O',1)]
    
    for i in range(size):
        for sign in range(2):
            toReturn = p[sign][1]
            
            row = all([m == p[sign][0] for m in subPlayGround[i,:]])
            cols = all([j == p[sign][0] for j in subPlayGround[:,i].transpose()])

            if row or cols:
                return toReturn,True
        
    return 0,False
    

In [5]:
def goal_test(playGround,connect,size):
    
    # get all windows having their size = connect . and check all of them one by one
    for newArray in getAllWindows(playGround,size,connect):
        
        # check for all rows and column
        rowColumn = checkRowColumn(newArray,connect) 
        if rowColumn[1]:
            return rowColumn[0]
                
        # checking diagnalOne and diagnalTwo
        p = [('x',-pow(10,connect-1)),('O',pow(10,connect-1))]
        for sign in range(2):
            toReturn = p[sign][1]
            diagnalOne = all([newArray[i,i] == p[sign][0] for i in range(connect)])
            diagnalTwo = all([newArray[i,connect-i-1] == p[sign][0] for i in range(connect)])
                
                
            if diagnalOne or diagnalTwo:
                return toReturn
                
    return 0


In [6]:
def getValidActions(playGround,size):
    validActions = []
    for i in range(size):
        for j in range(size):
            if playGround[i,j] == '-':
                validActions.append((i,j))
    return validActions


In [7]:
# def getHeauristicValue(vector,connect):
#     number = 1/10
#     values = []
#     for i in range(connect):
#         values.append(int(number*10))
#         number *= 10
        
#     prev = 0
#     contOne = 0
#     contOpp = 0
#     val = 0
#     for i in range(connect):
#         if vector[i] == 'x':
#             contOne = 0
#             if prev == 0 or prev == -1:
#                 val = -values[contOpp]
#                 contOpp += 1
#                 prev = -1
                    
#             else:
#                 val -= 1
#                 prev = -1
            
            
#         elif vector[i] == 'O':
#             contOpp = 0
#             if prev == 0 or prev == 1:
#                 val = values[contOne]
#                 contOne += 1
#                 prev = 1
#             else:
#                 val += 1
#                 prev = 1
        
    
#     return val

In [8]:
def getHeauristicValue(vector,connect):
    val = 0
    prev = '-'
    values = { '-':0, 'x':-1, 'O':1}
    if vector[0] == 'x':
        val = values['x']
        prev = 'x'
    elif vector[1] == 'O':
        val = values['O']
        prev = 'O'
    
    
    for i in range(1,connect):
        if vector[i] == 'x':
            if prev == 'x':
                val *= 10
            elif prev == 'O':
                return 0
            else:
                if val < 0:
                    val *= 10
                elif val > 0:
                    return 0
                else:
                    val = -1

        elif vector[i] == 'O':
            if prev == 'O':
                val *= 10
            elif prev == 'x':
                return 0
            else:
                if val > 0:
                    val *= 10
                elif val < 0:
                    return 0
                else:
                    val = 1
                    
    return val

In [9]:
def getVectorsFromWindow(window,connect):
    vectors = []
    diagnalOne = []
    diagnalTwo = []
    
    #add rows and columns
    for i in range(connect):
        vectors.append(window[i,:])
        vectors.append(window[:,i].transpose())
    
        #add diagnals
        diagnalOne.append(window[i,i])
        diagnalTwo.append(window[i,connect-i-1])
    
    vectors.append(diagnalOne)
    vectors.append(diagnalTwo)
    
    return vectors

    
    

In [10]:
def getHeauristic(playGround,size,connect,player,superval):
    heauristic = 0
    val = 0
    
#     if superval == 0:
    windows = getAllWindows(playGround,size,connect)
    for window in windows:
        vectors = getVectorsFromWindow(window,connect)
        for vector in vectors:
            heauristic += getHeauristicValue(vector,connect)
    
#     else:
#         superval = 0
#         actions = getValidActions(playGround,size)
#         val = inf
#         for action in actions:
#             newground = playGround.copy()
#             newground[action[0],action[1]] = 'x'
#             val = max(val,getHeauristic(playGround,size,connect,player,superval))
        
        
    
    return heauristic + val       

In [11]:
# returns row,column,value[userWon = -1,tie = 0, userLoose = 1]
def BasicMinimaxAlgo(playGround,noUserAlpha,noUserBeta,player,connect,size):
    # if user won
    goal = goal_test(playGround,connect,size)
    
    # goal = 0 means no one has won
    if goal != 0:
        return -1,-1,goal
    
    
    availableActions = getValidActions(playGround,size)
    
    # if no available actions -> means match draw
    if len(availableActions) == 0:
        return -1,-1,0
    
    # neither one won
    data = (-1,-1)
    
    # if player is max
    if player:
        val = -inf
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'O'
            a,b,value = BasicMinimaxAlgo(newplayGround,noUserAlpha,noUserBeta,False,connect,size)
            if value > val:
                val = value
                data = action
            
    # else if play is min
    else:
        val = inf
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'x'
            a,b,value = BasicMinimaxAlgo(newplayGround,noUserAlpha,noUserBeta,True,connect,size)
            if value < val :
                val = value
                data = action
    
#     print(data)
            
    return data[0],data[1],val           

In [12]:
def DepthBasicMinimaxAlgo(playGround,depth,noUserAlpha,noUserBeta,player,connect,size):

    goal = goal_test(playGround,connect,size)
    if goal != 0:
        return -1,-1,goal
    
    
    availableActions = getValidActions(playGround,size)
    
    
    if len(availableActions) == 0:
        return -1,-1,0
    
    
    if depth == 0:
        return -1,-1,getHeauristic(playGround,size,connect,player,1)
    
    # neither one won
    data = (-1,-1)
    
    # if player is max
    if player:
        val = -inf
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'O'
            a,b,value = DepthBasicMinimaxAlgo(newplayGround,depth,noUserAlpha,noUserBeta,False,connect,size)
            if value > val:
                val = value
                data = action
            
    # else if play is min
    else:
        val = inf
        depth -= 1
        
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'x'
            a,b,value = DepthBasicMinimaxAlgo(newplayGround,depth,noUserAlpha,noUserBeta,True,connect,size)
            if value < val :
                val = value
                data = action
    
#     print(data)
            
    return data[0],data[1],val           

In [13]:
def AlphaBetaMinimax(playGround,alpha,beta,player,connect,size):   
    
    goal = goal_test(playGround,connect,size)
    if goal != 0:
        return -1,-1,goal
    
    availableActions = getValidActions(playGround,size)
    
    # neither one won
    if len(availableActions) == 0:
        return -1,-1,0
    
    data = (-1,-1)
    
    # if player is max
    if player:
        val = -inf
        
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'O'
            a,b,value = AlphaBetaMinimax(newplayGround,alpha,beta,False,connect,size)
#             print(value)
            if value > val:
                val = value
                data = action
                
                
            alpha = max(alpha,val)
            if(alpha > beta):
                break
                
            
    # else if play is min
    else:
        val = inf
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'x'
            a,b,value = AlphaBetaMinimax(newplayGround,alpha,beta,True,connect,size)
            if value < val :
                val = value
                data = action
            
            beta = min(beta,val)
            if(alpha > beta):
                break
        
            
    return data[0],data[1],val     

In [14]:
def DepthAlphaBetaMinimax(playGround,depth,alpha,beta,player,connect,size):   

    goal = goal_test(playGround,connect,size)
    if goal != 0:
        return -1,-1,goal
    
    availableActions = getValidActions(playGround,size)
    
    # neither one won
    if len(availableActions) == 0:
        return -1,-1,0
    
    if depth == 0:
        return -1,-1,getHeauristic(playGround,size,connect,player,1)
    
    data = (-1,-1)
    
    # if player is max
    if player:
        val = -inf
        
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'O'
            a,b,value = DepthAlphaBetaMinimax(newplayGround,depth,alpha,beta,False,connect,size)
            if value > val:
                val = value
                data = action
                
                
            alpha = max(alpha,val)
            if(alpha > beta):
                break
                
            
    # else if play is min
    else:
        val = inf
        depth -= 1
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'x'
            a,b,value = DepthAlphaBetaMinimax(newplayGround,depth,alpha,beta,True,connect,size)
            if value < val :
                val = value
                data = action
            
            beta = min(beta,val)
            if(alpha > beta):
                break
        
            
        
#     print(data)    
    return data[0],data[1],val     

In [15]:
def EfficientMinimax(playGround,depth,alpha,beta,player,connect,size):   

    goal = goal_test(playGround,connect,size)
    if goal != 0:
        return -1,-1,goal
    
    availableActions = getValidActions(playGround,size)
    
    # neither one won
    if len(availableActions) == 0:
        return -1,-1,0
    
    if depth == 0:
        return -1,-1,getHeauristic(playGround,size,connect,player,1)
    
    data = (-1,-1)
    
    
    
    # if player is max
    if player:
        val = -inf
        
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'O'
            a,b,value = DepthAlphaBetaMinimax(newplayGround,depth,alpha,beta,False,connect,size)
            if value > val:
                val = value
                data = action
                
                
            alpha = max(alpha,val)
            if(alpha > beta):
                break
                
            
    # else if play is min
    else:
        val = inf
        depth -= 1
        for action in availableActions:
            newplayGround = playGround.copy()
            newplayGround[action[0],action[1]] = 'x'
            a,b,value = DepthAlphaBetaMinimax(newplayGround,depth,alpha,beta,True,connect,size)
            if value < val :
                val = value
                data = action
            
            beta = min(beta,val)
            if(alpha > beta):
                break
        
            
        
#     print(data)    
    return data[0],data[1],val     

In [16]:
def createPlayGround(size):
    playGround = []
    for i in range(size):
        playGround.append(['-'])
        for j in range(size-1):
            playGround[i].append('-')
            
    newplayGround = np.array(playGround)
    return newplayGround
    

In [17]:
class Agent:
    
    # initializing with all '-' environment
    def __init__(self,size,connect,depth):
        self.playGround = createPlayGround(size)        
        self.size = size
        self.depth = depth
        self.connect = connect
        self.computerChances = 0
        self.userChances = 0
        self.alpha = -inf
        self.beta = inf
        
        
    # if playGround is in other state
    def take_percept(self):
        self.playGround = createPlayGround(size)
        self.depth = depth
        self.computerChances = 0
        self.userChances = 0
        self.alpha = -inf
        self.beta = inf
        
    def setChoiceFunction(self,choice):
        isSet =  True
        function = DepthAlphaBetaMinimax
        
        if choice == 1:
            function = BasicMinimaxAlgo
            isSet = False
        elif choice == 2:
            function = AlphaBetaMinimax
            isSet = False
        elif choice == 3:
            function = DepthBasicMinimaxAlgo
            isSet = True
        elif choice == 4:
            function = DepthAlphaBetaMinimax
            isSet = True
#         else:
#             function = efficientMinimax
#             isSet = True
        
        return isSet,function
        
        
    # function to run
    # if computer came to know it will loose any way then it will put 'O' at the first place available for it.
    def playGame(self,user,choice):
        
        
        isSet,function = self.setChoiceFunction(choice)    
    
        gameOver = False
        while not gameOver:
            print(self.playGround,end='\n\n\n')
            if self.userChances + self.computerChances != (self.size*self.size) :

                if user:
                    ok = False
                    while not ok:
                        i,j = get_user_input(self.size)

                        if (i >= 0 and i < self.size ) and (j >= 0 and j< self.size) and (self.playGround[i,j] == '-'): 
                            self.playGround[i,j]='x'
                            self.userChances+=1


                            goal = goal_test(self.playGround,self.connect,self.size)

                            if goal == -1:
                                print('You Won')
                                return

                            ok = True
                        else:
                            print('\nwrong intput ')
            else:
                print('Match Tie')
                break


            if self.userChances + self.computerChances != (self.size*self.size) :
                user=True
                self.computerChances += 1
                newplayGround = self.playGround.copy()
                       
                
                # if user donot want to run depth limited functions
                if not isSet:
                    i,j,p = function(newplayGround,self.alpha,self.beta,True,self.connect,self.size)
                else:
                    i,j,p = function(newplayGround,self.depth,self.alpha,self.beta,True,self.connect,self.size)
                    
                
                
                self.playGround[i,j]='O'

                    
                        

                goal = goal_test(self.playGround,self.connect,self.size)
                if goal == 1:
                    print('you Loose')
                    break
                    


            else:
                print('Match Draw')
                break
    
             
                
    def showState(self):
        print(self.playGround)
    
    
        
    
    
    

In [20]:
print('Tic Toc Toe ')
chance = int(input("do you want odd chances or even chances enter 1 -> eventChances or 2 -> oddChances : "))
player = False if chance==1 else True


size,connect = map(int,input('enter comma separated size and points to connect : ').split(','))

# depth 1 means in tree :
#     level 1 : max
#     level 1 : min
#     level 0 : max  : now will run heauristic....
# according to this measurement it working quickly till depth = 3            
agent = Agent(size,connect,1)

playChoice = True
while playChoice:
    choice = int(input("""
    
    
Choose your algorithm :
1. BasicMinimaxAlgo
2. AlphaBetaMinimax
3. DepthLimitedMinimax
4. DepthLimitAlphaBetaMinimax
5. efficientMinimax


"""))
    
    agent.playGame(player,choice)
    playChoice = False
    
    if int(input('press One to play Again : ')) == 1:
        playChoice = True
        agent.take_percept()

print('Thanks ...')

Tic Toc Toe 
